# Feature Generation for Audio Data

### Below is a generalized script for feature extraction from a given set of .wav files. This script was executed for each sound and phrase audio file type (e.g., /a/ sounds, /i/ sounds, phrases). This resulted in features for each given sound across all intonation types. These features were later concatenated along recording_num.

### Import necessary libraries

In [ ]:
import urllib.parse
import parselmouth
import glob
import os.path
import librosa
import numpy as np
from parselmouth.praat import call
import pandas as pd
from urllib.parse import quote 

### Declare some initial variables, load helper dataframes, and declare arrays for features

In [ ]:
minimum_pitch, maximum_pitch, hop_length = 75, 600, 512
m_df = pd.read_csv(r'C:\Users\esabic\Desktop\Refactor_Attempts_11_4\All_M.csv')
f_df = pd.read_csv(r'C:\Users\esabic\Desktop\Refactor_Attempts_11_4\All_F.csv')
pathologisch_df = pd.read_csv(r'C:\Users\esabic\Desktop\Refactor_Attempts_11_4\All_Pathologisch.csv')
empty_array = pd.DataFrame(np.ones((0,139))) 
n_final_result = l_final_result = h_final_result = lhl_final_result = empty_array

### Loop through each .wav file located in a given directory. Call Praat for Praat features, use the Librosa package for Librosa features, append features to a given array based on intonation (n, l, h, lhl).

In [ ]:
for wave_file in glob.glob(r"C:\Users\esabic\Desktop\Data_Exporting\a_\*.wav"): 
    sex = age = target_label = 0 # reset
    # Sample Praat calls
    s = parselmouth.Sound(wave_file)        
    get_Power = call(s, "Get power...", 0.0, 0.0)
    hnr = call(s, "Get mean", 0, 0)
    pointProcess = call(s, "To PointProcess (periodic, cc)", minimum_pitch, maximum_pitch)
    Jitter_local = call(pointProcess, "Get jitter (local)...", 0, 0, .0001, .02, 1.3)
    Jitter_rap = call(pointProcess, "Get jitter (rap)...", 0, 0, .0001, .02, 1.3)
    Jitter_ppq5 = call(pointProcess, "Get jitter (ppq5)...", 0, 0, .0001, .02, 1.3)
    Jitter_ddp = call(pointProcess, "Get jitter (ddp)...", 0, 0, .0001, .02, 1.3)
    num_periods = call(pointProcess, "Get number of periods...", 0, 0, .0001, .02, 1.3)
    mean_period = call(pointProcess, "Get mean period...", 0, 0, .0001, .02, 1.3)
    stdev_period = call(pointProcess, "Get stdev period...", 0, 0, .0001, .02, 1.3)
    pitch = call(s, "To Pitch...", 0.0, minimum_pitch, maximum_pitch)
    meanF0 = call(pitch, "Get mean...", 0, 0, "Hertz") 
    stdevF0 = call(pitch, "Get standard deviation...", 0 ,0, "Hertz")    
    localShimmer =  call([s, pointProcess], "Get shimmer (local)...", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([s, pointProcess], "Get shimmer (local_dB)...", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([s, pointProcess], "Get shimmer (apq3)...", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([s, pointProcess], "Get shimmer (apq5)...", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([s, pointProcess], "Get shimmer (apq11)...", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([s, pointProcess], "Get shimmer (dda)...", 0, 0, 0.0001, 0.02, 1.3, 1.6)     
    get_Energy = call(s, "Get energy...", 0.0, 0.0)
    get_Energy_in_air = call(s, "Get energy in air")
    get_Power_in_air = call(s, "Get power in air")      
    get_mean_slope_Hz = call(pitch, "Get mean absolute slope...", "Hertz")
    get_mean_slope_Mel = call(pitch, "Get mean absolute slope...", "Mel")
    get_pitch_quantile = call(pitch, "Get quantile...", 0.0, 0.0, 0.50, "mel")    
    high_index = call(pointProcess, "Get high index...", 0.5)
    low_index = call(pointProcess, "Get low index...", 1)    
    Formant = call(s, "To Formant (burg)...", 0.0, 5.0, 5500, 0.025, 50.0)
    Spectrogram = call(s, "To Spectrogram...", 0.005, 5000, .002, 20, "Gaussian")    
    Spec_Power_1 = call(Spectrogram, "Get power at...", 0.25, 1000)
    Spec_Power_2 = call(Spectrogram, "Get power at...", 0.5, 1000)
    Spec_Power_3 = call(Spectrogram, "Get power at...", 0.75, 1000)
    Harmonicity = call(s, "To Harmonicity (cc)...", 0.01, 75.0, .1, 1)
    mean_harmonicity = call(Harmonicity, "Get mean...", 0, 0)
    stdev_harmonicity = call(Harmonicity, "Get standard deviation...", 0, 0)
    first_mean_harmonicity = call(Harmonicity, "Get mean...", 0, 0.5) 
    first_stdev_harmonicity = call(Harmonicity, "Get standard deviation...", 0, 0.5)       
    PowerCepstrogram = call(s, "To PowerCepstrogram...", 60, .002, 5000, 50)
    CPPS = call(PowerCepstrogram, "Get CPPS...", "yes", .02, .0005, 60, 330, .05, "Parabolic", .001, .05, "Exponential decay", "Least squares")    
    Autocorrelate = call(s, "Autocorrelate...", "peak 0.99", "zero")
    mean_autocorrelation = call(Autocorrelate, "Get mean...", 0, 0.0, 0.0)
    stdev_autocorrelation = call(Autocorrelate, "Get standard deviation", 0, 0.0, 0.0)
    rms_autocorrelation = call(Autocorrelate, "Get root-mean-square...", 0, 0.0)
    max_autocorrelation = call(Autocorrelate, "Get maximum", 0.0, 0.0, "Sinc70")
    intensity_autocorrelation = call(Autocorrelate, "Get intensity (dB)")
    
    # Sample Librosa Features   
    filename = wave_file
    y, sr = librosa.load(filename)       
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length = hop_length, n_mfcc = 12)
    
    if np.shape(mfcc)[1] < 13: # zero padding if needed
        how_many_needed = 12 - np.shape(mfcc)[1] 
        zero_padding = pd.DataFrame(np.zeros((12,how_many_needed)))
        mfcc = np.concatenate([mfcc, zero_padding], axis = 1)
    
    mfcc_delta = librosa.feature.delta(mfcc) 
    zc_rate = librosa.feature.zero_crossing_rate(y)
    S, phase = librosa.magphase(librosa.stft(y))
    spectral_roll_off = librosa.feature.spectral_rolloff(S = S, sr = sr)
    onset_env = librosa.onset.onset_strength(y = y, sr = sr)
    pitches, magnitudes = librosa.piptrack(y = y, sr = sr)
    rms_audio = librosa.feature.rms(y) 
    mean_rms_audio, stdev_rms_audio = np.mean(rms_audio), np.std(rms_audio)
    max_rms_audio, min_rms_audio = np.amax(rms_audio), np.amin(rms_audio)
    range_rms = max_rms_audio - min_rms_audio
    lpc = librosa.core.lpc(y, 12)      
    chroma_stft = librosa.feature.chroma_stft(y = y, sr = sr)
    
    split_filename = wave_file.split("\\")[-1].split("-")
    recording_number = int(split_filename[0])
    determine_pathologisch = pathologisch_df[pathologisch_df['pathological'] == recording_number]
    
    if len(determine_pathologisch) > 0:
        target_label = 1 # else it is a zero by default...
    
    determine_sex = m_df[m_df['individual'] == recording_number]
    
    if len(determine_sex) > 0: # if recording_number was found within m_keys, sex = 1
        sex = 1     
        
    if sex == 1: # M
        determine_age_group = m_df[m_df['individual'] == recording_number]
        if len(determine_age_group) == 0:
            age = '?' # use ? character to handle exeptions
        else:
            age = determine_age_group.iloc[0,1]
    else: # F
        determine_age_group = f_df[f_df['individual'] == recording_number]
        if len(determine_age_group) == 0:
            age = '?'
        else:
            age = determine_age_group.iloc[0,1]
    
    mean_mfcc, mean_delta = np.mean(mfcc, axis = 1), np.mean(mfcc_delta, axis = 1)
    mean_chroma = np.mean(chroma_stft, axis = 1)
    stdev_mfcc, stdev_delta = np.std(mfcc, axis = 1), np.std(mfcc_delta, axis = 1)
    stdev_chroma = np.std(chroma_stft, axis = 1)
    
    results = pd.DataFrame([get_Power, hnr, Jitter_local, Jitter_rap, Jitter_ppq5, Jitter_ddp, num_periods, mean_period,\
                   stdev_period, meanF0, stdevF0, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer,\
                   ddaShimmer, mean_mfcc[0], mean_mfcc[1], mean_mfcc[2], mean_mfcc[3], \
                   mean_mfcc[4], mean_mfcc[5], mean_mfcc[6], mean_mfcc[7], \
                   mean_mfcc[8], mean_mfcc[9], mean_mfcc[10], mean_mfcc[11], \
                   mean_delta[0],mean_delta[1],mean_delta[2],mean_delta[3],\
                   mean_delta[4],mean_delta[5],mean_delta[6],mean_delta[7],\
                   mean_delta[8],mean_delta[9],mean_delta[10],mean_delta[11],\
                   stdev_mfcc[0], stdev_mfcc[1], stdev_mfcc[2], stdev_mfcc[3],
                   stdev_mfcc[4], stdev_mfcc[5], stdev_mfcc[6], stdev_mfcc[7],\
                   np.mean(zc_rate), np.std(zc_rate), np.mean(spectral_roll_off), np.std(spectral_roll_off), np.mean(onset_env),\
                   np.std(onset_env), np.std(pitches), np.std(magnitudes), np.mean(magnitudes), mean_rms_audio,\
                   stdev_rms_audio, max_rms_audio, min_rms_audio, range_rms, lpc[1], lpc[2], lpc[3], lpc[4], lpc[5], lpc[6], lpc[7], lpc[8],\
                   lpc[9], lpc[10], lpc[11], recording_number, sex, age, target_label, stdev_delta[0], stdev_delta[1], stdev_delta[2], stdev_delta[3],\
                   stdev_delta[4], stdev_delta[5], stdev_delta[6], stdev_delta[7], stdev_delta[8], stdev_delta[9], stdev_delta[10], \
                   stdev_delta[11], stdev_mfcc[8], stdev_mfcc[9], stdev_mfcc[10], stdev_mfcc[11], mean_chroma[0], mean_chroma[1], mean_chroma[2],\
                   mean_chroma[3], mean_chroma[4], mean_chroma[5], mean_chroma[6], mean_chroma[7], mean_chroma[8], mean_chroma[9],\
                   mean_chroma[10], mean_chroma[11], stdev_chroma[0],stdev_chroma[1],stdev_chroma[2],\
                   stdev_chroma[3],stdev_chroma[4],stdev_chroma[5],stdev_chroma[6],stdev_chroma[7],stdev_chroma[8],\
                   stdev_chroma[9],stdev_chroma[10],stdev_chroma[11], get_Energy, get_Energy_in_air, get_Power_in_air, get_mean_slope_Hz, \
                   get_mean_slope_Mel, get_pitch_quantile, high_index, low_index, Spec_Power_1, Spec_Power_2, Spec_Power_3, mean_harmonicity, stdev_harmonicity, first_mean_harmonicity,\
                   first_stdev_harmonicity, CPPS, mean_autocorrelation, stdev_autocorrelation, rms_autocorrelation, max_autocorrelation, intensity_autocorrelation]).T 
        
    speech_sound = split_filename[1].split('.')[0] # structure for l,h,n, and lhl. This variable can be hard coded for phrase clips
    
    if speech_sound == 'l':
        l_final_result = pd.concat([l_final_result, results], axis = 0)
    elif speech_sound == 'h':
        h_final_result = pd.concat([h_final_result, results], axis = 0)
    elif speech_sound == 'n':
        n_final_result = pd.concat([n_final_result, results], axis = 0)
    elif speech_sound == 'lhl':
        lhl_final_result = pd.concat([lhl_final_result, results], axis = 0)

### Rename columns in preparation for data export.

In [ ]:
l_final_result.columns = h_final_result.columns = n_final_result.columns = lhl_final_result.columns = ['Power','hnr','Jitter_loc','Jitter_rap','Jitter_ppq5','Jitter_ddp','num_periods','mean_period',\
                       'stdev_period','meanF0','stdevF0','localShimmer','localdbShimmer','apq3Shimmer','aqpq5Shimmer','apq11Shimmer',\
                       'ddaShimmer','mean_mfcc_0','mean_mfcc_1','mean_mfcc_2','mean_mfcc_3',\
                       'mean_mfcc_4','mean_mfcc_5','mean_mfcc_6','mean_mfcc_7',\
                       'mean_mfcc_8','mean_mfcc_9','mean_mfcc_10','mean_mfcc_11',\
                       'mean_mfcc_delta_0','mean_mfcc_delta_1','mean_mfcc_delta_2','mean_mfcc_delta_3','mean_mfcc_delta_4',\
                       'mean_mfcc_delta_5','mean_mfcc_delta_6','mean_mfcc_delta_7',\
                       'mean_mfcc_delta_8','mean_mfcc_delta_9','mean_mfcc_delta_10','mean_fcc_delta_11',\
                       'stdev_mfcc_0','stdev_mfcc_1','stdev_mfcc_2','stdev_mfcc_3', 
                       'stdev_mfcc_4','stdev_mfcc_5','stdev_mfcc_6','stdev_mfcc_7',                           
                       'mean_zc','stdev_zc','mean_spectral_roll_off','stdev_spectral_roll_off','mean_onset_env',             
                       'stdev_onset_env','stdev_pitches','stdev_magnitudes','mean_magnitudes','mean_rms_audio',                          
                       'stdev_rms_audio','max_rms','min_rms','range_rms','lpc1','lpc2','lpc3','lpc4','lpc5','lpc6','lpc7','lpc8',\
                       'lpc9','lpc10','lpc11','recording_num','sex','age','y','sd_mfcc_delta_0','sd_mfcc_delta_1','sd_mfcc_delta_2','sd_mfcc_delta_3',\
                       'sd_mfcc_delta_4', 'sd_mfcc_delta_5', 'sd_mfcc_delta_6', 'sd_mfcc_delta_7', 'sd_mfcc_delta_8', 'sd_mfcc_delta_9','sd_mfcc_delta_10', 'sd_mfcc_delta_11',
                       'stdev_mfcc_8','stdev_mfcc_9','stdev_mfcc_10','stdev_mfcc_11',                           
                       'mean_chroma_0','mean_chroma_1','mean_chroma_2','mean_chroma_3','mean_chroma_4','mean_chroma_5','mean_chroma_6','mean_chroma_7',
                       'mean_chroma_8', 'mean_chroma_9', 'mean_chroma_10', 'mean_chroma_11', 'sd_chroma_0','sd_chroma_1','sd_chroma_2','sd_chroma_3',
                       'sd_chroma_4','sd_chroma_5','sd_chroma_6','sd_chroma_7','sd_chroma_8','sd_chroma_9','sd_chroma_10','sd_chroma_11',        
                       'Energy', 'Energy_in_air', 'Power_in_air', 'mean_slope_Hz', 'mean_slope_Mel','pitch_quantile','high_index','low_index',
                       'Spec_Power_1','Spec_Power_2','Spec_Power_3','mean_harmony','sd_harmony','first_mean_harmony','first_sd_harmony','CPPS','mean_autocorrelation',
                       'sd_autocorrelation','rms_autocorrelation','max_autocorrelation','intensity_autocorrelation']